In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

logger = getLogger(__name__)

DIR = '../result_tmp/'

log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s][%(funcName)s] %(message)s ')
handler = StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(log_fmt)
logger.addHandler(handler)

handler = FileHandler(DIR + 'train.py.log', 'a')
handler.setLevel(DEBUG)
handler.setFormatter(log_fmt)
logger.setLevel(DEBUG)
logger.addHandler(handler)

logger.info('start')

dtypes = {'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}

train = pd.read_csv('../../input/train_small.csv',  dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 26565) #Skip dates before 2016-08-01
                    )
logger.info('load data successful')

train = train.loc[(train.store_nbr == 9), ]

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek


# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)

logger.info('reindex train data')

train.head(10)

2017-11-29 06:30:31,044 __main__ 22 [INFO][<module>] start 
2017-11-29 06:30:31,060 __main__ 29 [INFO][<module>] load data successful 
2017-11-29 06:30:31,131 __main__ 51 [INFO][<module>] reindex train data 


,,,unit_sales,dow
date,store_nbr,item_nbr,,
2016-08-01,9,103501,2.079442,0
2016-08-02,9,103501,2.639057,1
2016-08-04,9,103501,1.791759,3
2016-08-05,9,103501,1.945910,4
2016-08-06,9,103501,1.791759,5
2016-08-07,9,103501,2.197225,6
2016-08-08,9,103501,1.386294,0
2016-08-09,9,103501,1.791759,1
2016-08-10,9,103501,1.098612,2


In [2]:


train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
train.reset_index(inplace=True) # reset index and restoring unique columns
lastdate = train.iloc[train.shape[0]-1].date


In [3]:
#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)
ma_dw.head(5)

,item_nbr,store_nbr,dow,madw
0,103501,9,0,1.713771
1,103501,9,1,1.790115
2,103501,9,2,1.680478
3,103501,9,3,1.687456
4,103501,9,4,1.712395


In [4]:
train

,date,store_nbr,item_nbr,unit_sales,dow
0,2016-08-01,9,103501,2.079442,0
1,2016-08-02,9,103501,2.639057,1
2,2016-08-04,9,103501,1.791759,3
3,2016-08-05,9,103501,1.945910,4
4,2016-08-06,9,103501,1.791759,5
5,2016-08-07,9,103501,2.197225,6
6,2016-08-08,9,103501,1.386294,0
7,2016-08-09,9,103501,1.791759,1
8,2016-08-10,9,103501,1.098612,2
9,2016-08-11,9,103501,1.098612,3


In [5]:
#Moving Averages
logger.info('start calcualte moving average')
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg,train

ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)
ma_is.head(5)

2017-11-29 06:30:31,278 __main__ 2 [INFO][<module>] start calcualte moving average 


,item_nbr,store_nbr,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais
0,103501,9,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809


In [6]:
#Load test
logger.info('load test data')
test = pd.read_csv('../../input/test_small.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

del ma_is, ma_wk, ma_dw

#Forecasting Test
test['unit_sales'] = test.mais
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values

#50% more for promotion items
test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

test = test.loc[(test.store_nbr == 9), ]
test.head(10)

2017-11-29 06:30:31,394 __main__ 2 [INFO][<module>] load test data 


,id,date,store_nbr,item_nbr,onpromotion,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais,mawk,madw,unit_sales
8,125528250,2017-08-16,9,103501,False,2,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.680478,3.732535
62,125738904,2017-08-17,9,103501,False,3,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.687456,3.763179
116,125949558,2017-08-18,9,103501,False,4,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.712395,3.874338
170,126160212,2017-08-19,9,103501,False,5,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.978886,5.236955
224,126370866,2017-08-20,9,103501,False,6,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,2.171514,6.453429
278,126581520,2017-08-21,9,103501,False,0,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.713771,3.880545
332,126792174,2017-08-22,9,103501,False,1,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.790115,4.237668
386,127002828,2017-08-23,9,103501,False,2,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.680478,3.732535
440,127213482,2017-08-24,9,103501,False,3,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.687456,3.763179
494,127424136,2017-08-25,9,103501,False,4,1.816616,1.736714,1.700783,1.627899,1.664834,1.54108,1.656604,2.079442,1.682809,1.819231,1.712395,3.874338
